### O METAR, sigla de Meteorological Aerodrome Report, é o informe meteorológico regular de aeródromo e contém as seguintes informações na sequência:

- Grupos de identificação;  
- Vento à superfície;  
- Visibilidade horizontal; 
- Alcance visual na pista (quando houver); 
- Tempo presente; 
- Nuvens (ou visibilidade vertical, se for o caso); 
- Temperaturas do ar e do ponto de orvalho; 
- Pressão atmosférica (QNH); e  
- Informações suplementares de inclusão condicional sobre tempo recente, cortante do vento, temperatura da superfície do mar, estado do mar e, por Acordo Regional de Navegação Aérea, o estado da pista.

https://ajuda.decea.mil.br/base-de-conhecimento/como-decodificar-o-metar-e-o-speci/


pip install python-metar


In [2]:
from metar import Metar
from datetime import datetime, timedelta
from util.utils import *
import pandas as pd
import re

df_metar = pd.read_csv("../data/processed_data/metaf_bronze.csv")


In [1]:
def interpretaMetaR(metarBruto):
    if("METAF" in metarBruto):
        metarBruto = metarBruto.replace('METAF', 'METAR')
        metarBruto = metarBruto.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarBruto[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
        



In [43]:
resultado_df =  df_metar['metaf'].apply(interpretaMetaR)
resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['station', 'type', 'metar_hora_formated', 'station','dew_point','windDirection','windVelocity','visibility','pressure','weather','sky','skyFeet'] )
metaR_bronze = pd.concat([df_metar, resultado_df], axis=1)


In [44]:
direction_to_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': None,
    'variable': None,
}

metaR_bronze['windDirection'] = metaR_bronze['windDirection'].map(direction_to_degrees)

In [45]:
metaR_bronze = metaR_bronze.sort_values(by=['aero','metar_hora_formated'])
metaR_bronze = metaR_bronze.drop_duplicates()
metaR_bronze = metaR_bronze.dropna()


In [47]:
def to_dummies(df):
    df_Dummies = pd.get_dummies(df['sky'])
    df_final = pd.concat([df, df_Dummies], axis=1)

    return df_final


In [48]:
def defineIndex(df):
    df = df.sort_values(by='hora_formatted')
    df.set_index('hora_formatted', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df
    
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []

    for coluna in colunas:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
        df_resampled_temp = df[coluna].resample('30T').mean()
        df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
        tempSeries = calculate_average(df_resampled_temp_filled)
        listSeries.append(tempSeries)


    dataframe_resultante = pd.concat(listSeries, axis=1)

    dataframe_resultante.columns = colunas

    return dataframe_resultante





In [49]:
metaR_silver = defineIndex(metaR_silver)

In [50]:
mediaMetaR_silver = pipemediaTempoMetar(metaR_silver,['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
metaR_silver = metaR_silver.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)

C:\Users\Gaspar\AppData\Local\Temp\ipykernel_15136\860116946.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')


In [61]:
novo_dataframe = pd.merge(metaR_silver, mediaMetaR_silver, left_index=True, right_index=True)


In [52]:
novo_dataframe = novo_dataframe.sort_values(by=['hora','aero'])

In [63]:
novo_dataframe

,hora,metaf,aero,station,type,metar_hora_formated,station,a few clouds,a few cumulonimbus,broken clouds,overcast,scattered clouds,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
hora_formatted,,,,,,,,,,,,,,,,,,,,
2023-02-28 21:00:00,1677628800000,METAF SBGR 010000Z 11006KT 3000 BR OVC033...,SBGR,SBGR,routine report (automatic report),Thu Aug 31 21:00:00 2023,20.0,False,False,False,True,False,19.500000,101.25,6.0,4500.000000,1018.000000,NaN,NaN,2475.000000
2023-02-28 21:00:00,1677628800000,METAF SBGL 010000Z 07002KT 9000 - RA BKN018 ...,SBGL,SBGL,routine report (automatic report),Thu Aug 31 21:00:00 2023,26.0,False,False,True,False,False,19.500000,101.25,6.0,4500.000000,1018.000000,NaN,NaN,2475.000000
2023-02-28 21:00:00,1677628800000,METAF SBKP 010000Z 11011KT 4000 BR BKN015...,SBKP,SBKP,routine report (automatic report),Thu Aug 31 21:00:00 2023,22.0,False,False,True,False,False,19.500000,101.25,6.0,4500.000000,1018.000000,NaN,NaN,2475.000000
2023-02-28 21:00:00,1677628800000,METAF SBSP 010000Z 11005KT 2000 BR OVC033...,SBSP,SBSP,routine report (automatic report),Thu Aug 31 21:00:00 2023,20.0,False,False,False,True,False,19.500000,101.25,6.0,4500.000000,1018.000000,NaN,NaN,2475.000000
2023-02-28 22:00:00,1677632400000,METAF SBGR 010100Z 11005KT 3000 BR OVC033...,SBGR,SBGR,"routine report, cycle 1 (automatic report)",Thu Aug 31 22:00:00 2023,20.0,False,False,False,True,False,18.666667,105.00,6.0,2666.666667,1018.666667,NaN,NaN,2400.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-01 18:00:00,1680382800000,METAF SBSP 012100Z 15010KT 4000 - RABR OVC03...,SBSP,SBSP,"routine report, cycle 21 (automatic report)",Fri Sep 01 18:00:00 2023,22.0,False,False,False,True,False,18.666667,135.00,9.0,3666.666667,1013.333333,NaN,NaN,2533.333333
2023-04-01 18:00:00,1680382800000,METAF SBCT 012100Z 10008KT 1000 - RABR OVC03...,SBCT,SBCT,"routine report, cycle 21 (automatic report)",Fri Sep 01 18:00:00 2023,19.0,False,False,False,True,False,18.666667,135.00,9.0,3666.666667,1013.333333,NaN,NaN,2533.333333
2023-04-01 19:00:00,1680386400000,METAF SBSP 012200Z 15010KT 3000 RABR OVC03...,SBSP,SBSP,"routine report, cycle 22 (automatic report)",Fri Sep 01 19:00:00 2023,21.0,False,False,False,True,False,18.000000,135.00,9.0,2666.666667,1013.333333,NaN,NaN,3200.000000
